<a href="https://colab.research.google.com/github/shoma3571/recommend-engine/blob/main/chapter2/1_CollaborativeFiltering_MovieLens100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

必要なライブラリのインポート

In [12]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

各ユーザーに類似ユーザートップ10を計算する

In [13]:
topk = 10
rank_list = [i+1 for i in range(topk)]

MovieLens 100k のデータを取得

In [14]:
ML100K_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
dataset = pd.read_csv(
    ML100K_URL,
    names = ["user_id", "item_id", "rating", "timestamp"],
    sep = "\t"
)

## User2User
ユーザー基点のスパースマトリクスに変換

In [19]:
# スパースマトリクスに変換
user_data = csr_matrix(
    (dataset.rating,
    (dataset.user_id, dataset.item_id))
)

In [20]:
# コサイン類似度を求める
# コサイン類似度は2つのベクトルがどれくらい同じ向きを向いているのかを表す指標
user_similarity = cosine_similarity(user_data)

各ユーザーに類似度の高いユーザートップ10を絞りこむ

In [21]:
# ユニークなユーザーIDのリストを作成する
uq_users = np.sort(dataset.user_id.unique().tolist())

# 類似ユーザー情報を格納するテーブルを作成する
df_similar_user_list = pd.DataFrame(
    columns = [
        "base_user_id", # 対象ユーザー
        "similar_user_id", # 類似ユーザー
        "cosine_score", # コサイン類似度
        "rank" # 順位
    ]
)


# 各ユーザー(対象ユーザー)に対して類似ユーザーを絞り込む
for user_id in uq_users:
  # 対象ユーザーから見た類似度スコア情報を取得
  similar_score = user_similarity[user_id]

  df_similar_user = pd.DataFrame(
      columns = [
        "base_user_id", # 対象ユーザー
        "similar_user_id", # 類似ユーザー
        "cosine_score", # コサイン類似度
        "rank" # 順位
    ]
  )
  df_similar_user['base_user_id'] = [user_id] * topk
  # コサイン類似度からトップ10類似ユーザーを取得
  df_similar_user['similar_user_id'] = np.argsort(similar_score)[::-1][1:topk+1]
  # トップ10の類似ユーザーのコサイン類似度を取得
  df_similar_user['cosine_score'] = np.sort(similar_score)[::-1][1:topk+1]


  # 全体テーブルに格納
  df_similar_user['rank'] = rank_list
  df_similar_user_list = df_similar_user_list.append(
      df_similar_user,
      ignore_index=True
  )

df_similar_user_list

,base_user_id,similar_user_id,cosine_score,rank
0,1,916,0.569066,1
1,1,864,0.547548,2
2,1,268,0.542077,3
3,1,92,0.540534,4
4,1,435,0.538665,5
...,...,...,...,...
9425,943,276,0.498243,6
9426,943,709,0.493219,7
9427,943,586,0.491972,8
9428,943,472,0.488882,9


## Item2Item
アイテム基点のスパースマトリクスに変換

In [22]:
item_data = csr_matrix(
    (dataset.rating,
    (dataset.item_id, dataset.user_id))
)

In [23]:
# コサイン類似度
item_similarity = cosine_similarity(item_data)

In [24]:
# ユニークなアイテムIDのリストを作成する
uq_items = np.sort(dataset.item_id.unique().tolist())

# 類似アイテム情報を格納するテーブルを作成する
df_similar_item_list = pd.DataFrame(
    columns = [
        'base_item_id', # 対象アイテム
        'similar_item_id', # 類似アイテム
        'cosine_score', # コサイン類似度
        'rank' # 順位
    ]
)


# 各アイテムに対して類似アイテムを絞り込む
for item_id in uq_items:
  similar_score = item_similarity[item_id]

  df_similar_item = pd.DataFrame()
  df_similar_item['base_item_id'] = [item_id] * topk
  df_similar_item['similar_item_id'] = np.argsort(similar_score)[::-1][1:topk+1]
  df_similar_item['cosine_score'] = np.sort(similar_score)[::-1][1:topk+1]
  df_similar_item['rank'] = rank_list

  df_similar_item_list = df_similar_item_list.append(
      df_similar_item,
      ignore_index=True
  )

df_similar_item_list

,base_item_id,similar_item_id,cosine_score,rank
0,1,50,0.734572,1
1,1,181,0.699925,2
2,1,121,0.689786,3
3,1,117,0.664555,4
4,1,405,0.641322,5
...,...,...,...,...
16815,1682,1268,0.266207,6
16816,1682,1113,0.248069,7
16817,1682,1428,0.231455,8
16818,1682,960,0.229794,9
